# 환율 데이터로 더 스마트한 경제 생활하기
[배경 이야기] <br>
직장인 A씨는 해외 직구를 즐겨 하며, 여행도 자주 다니는 편입니다. 최근 친구들과 동남아로 여행을 계획하고 있는데, 환율이 자주 변동되어 언제 환전을 해야 할지 고민이 많습니다. 마침 직장 동료 B씨는 해외 주식에 투자 중이라, 환율의 변화가 수익률에 직접적인 영향을 미치고 있어 매일 환율 정보를 꼼꼼히 살펴보고 있습니다.
 
하지만 매일 환율 데이터를 수동으로 확인하는 것은 너무 번거롭고, 적시에 정보를 놓칠 때가 많았습니다. 이에 A씨와 B씨는 네이버 금융 환율 정보를 자동으로 수집해, 실시간으로 중요한 환율 데이터를 분석하고 활용하기로 결정했습니다.
 
### BX : 네이버 금융의 신뢰성 있는 데이터
- 네이버 금융은 환율, 주식, 채권 등 다양한 경제 정보를 신뢰성 있게 제공하는 플랫폼입니다.
 
### CX : 사용자의 편의성을 위한 맞춤형 데이터
- 매일 네이버 금융 사이트를 직접 확인하는 번거로움을 줄이고, 환율 변동을 바로 확인할 수 있도록 크롤링을 통해 데이터를 수집하고 시각화합니다.
 
### DX : 자동화된 환율 데이터 수집과 분석
- 기술을 활용해 데이터를 수집, 분석, 시각화하면 효율적으로 환율 정보를 활용할 수 있습니다.

In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd

In [27]:
# 1. 사이트 요청
res = req.get("https://finance.naver.com/marketindex/")
res

<Response [200]>

In [29]:
# 2. 데이터 파싱
soup = bs(res.text, "lxml")

# 3. 통화명 수집
soup.select("body > div > table > tbody > tr:nth-child(1) > td.tit > a")

[]

### 응답코드는 200인데 데이터가 나오지 않는다면?
- 응답코드 200의 의미는 통신에 성공
- But! 원하는 태그의 결과가 나오지 않는 경우가 발생
- Why? 화면상에 눈으로는 보이지만, 실제 데이터는 다른 주소에 저장이 되어있는 경우
- iframe태그 : 다른 서버에 저장된 파일을 빌려다가 보여주기 위한 태그
- How? 접근하고자 하는 데이터가 있는 주소로 요청을 보내야 함

In [35]:
# iframe태그 접근
soup.select("#frame_ex1")  #--> list 형식

[<iframe frameborder="0" height="1800px" id="frame_ex1" marginheight="0" marginwidth="0" scrolling="no" src="/marketindex/exchangeList.naver" title="환전 고시 환율" width="100%"></iframe>]

In [37]:
# 한 개만 필요한 경우에는 select_one함수를 활용
# Why? select로 수집하면 데이터가 한개여도 무조건 리스트가 반환됨
iframe = soup.select_one("#frame_ex1")
iframe

<iframe frameborder="0" height="1800px" id="frame_ex1" marginheight="0" marginwidth="0" scrolling="no" src="/marketindex/exchangeList.naver" title="환전 고시 환율" width="100%"></iframe>

In [41]:
url = iframe['src']
url

'/marketindex/exchangeList.naver'

In [45]:
real_url = "https://finance.naver.com"+url

# 실제 데이터가 존재하는 곳으로 재요청
res = req.get(real_url)
res

<Response [200]>

In [47]:
soup = bs(res.text, "lxml")
soup

<html lang="ko">
<head>
<title>네이버페이 증권</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20241226190829/css/finance.css" rel="stylesheet" type="text/css"/>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20241226190829/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20241226190829/js/ntm.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20241226190829/js/nLog.js" type="text/javascript"></script>
</head>
<body>
<div class="tbl_area">
<table border="1" class="tbl_exchange" summary="환전 고시 환율 리스트">
<caption>환전 고시 환율</caption>
<colgroup>
<col width="162"/>
<col width="92"/>
<col width="92"/>
<col width="92"/>
<col width="93"/>
<col width="92"/>
<col width="90

In [59]:
name = soup.select("body > div > table > tbody > tr:nth-child(1) > td.tit > a")
name[0].text

'\n\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t미국 USD\n\t\t\t\t\n\t\t\t\t'

In [61]:
name[0].text.strip()

'미국 USD'

In [65]:
# 컨텐츠만 저장
name_list = []
for i in name:
    name_list.append(i.text.strip())

name_list

['미국 USD']

### 이번파일 정리
- req로 요청을 보냈을 때 응답코드가 200이 나와도 모든 데이터가 있다고 보면 안됨
- 조회하고자 하는 데이터가 추출되지 않는다면, 반드시 iframe을 의심해봐라
- iframe을 src속성에 진짜 데이터가 존재하는 주소를 가지고 있음
- But! 보안에 신경쓰는 개발자는 src에 값을 숨기는 경우도 존재함
- src에 있는 경로를 활용해서 절대경로로 제작 후 -> 새롭게 요청
- req로 불가능하다면, selenium(자동화)라이브러리로 해결이 가능

-----

## 추가 아이디어
- 환율 데이터와 함께 주요 경제 지표(금값, 유가 등) 데이터를 수집해 연관성을 분석.
- 변동성이 큰 통화에 대해 예측 모델을 구축해 "환율 경고 알림" 서비스 구현.
- 데이터를 시각화하여 하루, 주간, 월간 변동 추세를 한눈에 볼 수 있는 대시보드 제작.
상황에 맞는 메뉴 있음